In [3]:
cd /content/drive/MyDrive/mygithub/pytorchForAudioPractice

/content/drive/.shortcut-targets-by-id/1j0tKfNJ74iEAtyLmY4PY6L2fktF9jL1I/mygithub/pytorchForAudioPractice


In [4]:
!pip install torch torchaudio torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.autograd.profiler as profiler

In [6]:
def download_mnist_datasets():
  train_data = datasets.MNIST(train=True, download=True, transform=ToTensor(), root="data")
  return train_data


In [7]:
from torch.nn.modules.linear import Linear
class FeedForwardNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.denseLayers = nn.Sequential(
        nn.Linear(28*28,256), # image size 28x28
        nn.ReLU(),
        nn.Linear(256,10) # 10 classes 0 to 9 
    )
    self.activation = nn.Softmax(dim=1)

  def forward(self, inputData):

    flattenedData = self.flatten(inputData)
    logits = self.denseLayers(flattenedData)
    predictions = self.activation(logits)

    return predictions

In [9]:
def trainOneEpoch(model, dataLoader, lossFn, optimiser, device):


  for input, target in dataLoader:

    input, target = input.to(device), target.to(device)

    #calculate loss
    predication = model(input)

    loss = lossFn(predication, target)

    # backpropagation and weight update

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

  print(f"loss: {loss.item()}")

In [10]:
def train(model, dataLoader, lossFn, optimiser, device, epochs):
  for i in range(epochs):

    print(f"Epoch {i+1}")
    trainOneEpoch(model, dataLoader, lossFn, optimiser, device)
    print("---------------------------")
  
  print("Finished training")

In [12]:
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 10

# Download data
train_data = download_mnist_datasets()

# data load
dataLoader = DataLoader(train_data, batch_size=BATCH_SIZE)

# Build Model

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device}")

model = FeedForwardNN().to(device)

# Train

lossFn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


train(model, dataLoader, lossFn, optimiser, device, EPOCHS)

# Save Model

torch.save(model.state_dict(),"modelNN.pth")
print("Trained feed forward net saved at modelNN.pth")



Using cuda
Epoch 1
loss: 1.5150885581970215
---------------------------
Epoch 2
loss: 1.5025283098220825
---------------------------
Epoch 3
loss: 1.494546890258789
---------------------------
Epoch 4
loss: 1.4855036735534668
---------------------------
Epoch 5
loss: 1.4817465543746948
---------------------------
Epoch 6
loss: 1.4765969514846802
---------------------------
Epoch 7
loss: 1.4747270345687866
---------------------------
Epoch 8
loss: 1.4726533889770508
---------------------------
Epoch 9
loss: 1.4732893705368042
---------------------------
Epoch 10
loss: 1.472215175628662
---------------------------
Finished training
Trained feed forward net saved at modelNN.pth
